# Import Libraries

In [1]:
from swarmintelligence import GreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = {
    'maxIteration': 100,
    'wolfSize': 25
}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo = GreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs, best_thresholds = gwo.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(gwo.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(gwo.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                gwo_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions'].append(mean_regions)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['Fitness'].append(fitness_values)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
12.211474090566005
Mean Fitness = 12.211474090566007
	 AirplaneF16.tiff | Threshold =  3
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
15.503893789532656
1

21.561730298487927
21.576238869444282
21.566764278853796
21.57237078643271
Mean Fitness = 21.547170118350337
	 Mandrill.tiff | Threshold =  2
12.217727299731392
12.217754102632629
12.217727299731392
12.217754102632629
12.217701316049435
12.217727299731392
12.217727299731392
12.217727299731392
12.21727770545774
12.217754102632629
12.217727299731392
12.217727299731392
12.217727299731392
12.217727299731392
12.217727299731392
12.217754102632629
12.217727299731392
12.217727299731392
12.217754102632629
12.217727299731392
12.217727299731392
12.217754102632629
12.217727299731392
12.217701316049435
12.217701316049435
12.217632467487256
12.217701316049435
12.217754102632629
12.217727299731392
12.217754102632629
Mean Fitness = 12.217712835130204
	 Mandrill.tiff | Threshold =  3
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.279196147154849
15.2791961471

21.02488614810324
21.024432179341293
21.025601459050485
20.964243312635183
21.02488614810324
21.02488614810324
20.908252653060835
21.016580502964178
21.02488614810324
21.025601459050485
20.91601612149963
21.0242408799802
Mean Fitness = 20.987219338392382


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[71, 173]",2.495186,12.211474,2806.376312,52.975242,13.649345,0.761347,0.921776,"[[71, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0.9304120540618896, 1.328341007232666, 0.9778...","[12.211474090566005, 12.211474090566005, 12.21...","[2806.3763122558594, 2806.3763122558594, 2806....","[52.97524244640943, 52.97524244640943, 52.9752...","[13.64934454910294, 13.64934454910294, 13.6493...","[0.761346872735969, 0.761346872735969, 0.76134...","[0.9217762274412886, 0.9217762274412886, 0.921..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[69, 127, 183]",4.098413,15.503873,1994.788860,44.663059,15.131834,0.798135,0.949486,"[[69, 255, 255, 255, 183, 255, 255, 255, 255, ...","[5.157691955566406, 5.594018936157227, 3.87682...","[15.503893789532656, 15.503893789532656, 15.50...","[1994.712142944336, 1994.712142944336, 1994.71...","[44.66220038180313, 44.66220038180313, 44.6622...","[15.132001293919405, 15.132001293919405, 15.13...","[0.7981453760593136, 0.7981453760593136, 0.798...","[0.949480595306752, 0.949480595306752, 0.94948..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[66, 105, 145, 185]",4.412756,18.311407,1822.628710,42.692251,15.523824,0.812936,0.956895,"[[66, 255, 255, 255, 185, 255, 255, 255, 255, ...","[3.245598316192627, 4.998659372329712, 4.04133...","[18.311946771930018, 18.311946771930018, 18.31...","[1821.50244140625, 1821.50244140625, 1820.2124...","[42.679063267675524, 42.679063267675524, 42.66...","[15.52650603193598, 15.52650603193598, 15.5295...","[0.8129814935206013, 0.8129814935206013, 0.813...","[0.9568940992920079, 0.9568940992920079, 0.956..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[59, 89, 123, 156, 188]",3.911308,20.901962,1710.462949,41.357118,15.799929,0.813295,0.961305,"[[89, 255, 255, 188, 188, 188, 255, 255, 255, ...","[3.8039355278015137, 4.260148763656616, 3.0532...","[20.90731517318557, 20.904275293937772, 20.908...","[1692.931224822998, 1696.6587181091309, 1697.3...","[41.14524547044285, 41.19051733238041, 41.1984...","[15.84441045574428, 15.834858678418335, 15.833...","[0.812399449605161, 0.8119701777787723, 0.8118...","[0.9614733216725029, 0.9617294246001322, 0.961..."
4,Lena.png,2,kapur_entropy,max,"[97, 164]",3.826801,12.346365,2044.689766,45.218246,15.024529,0.630764,0.891673,"[[164, 164, 164, 164, 164, 164, 164, 164, 255,...","[4.149683952331543, 3.2339608669281006, 3.2934...","[12.346365101175513, 12.346365101175513, 12.34...","[2044.6897659301758, 2044.6897659301758, 2044....","[45.21824594044063, 45.21824594044063, 45.2182...","[15.02452937602985, 15.02452937602985, 15.0245...","[0.6307635952929036, 0.6307635952929036, 0.630...","[0.8916733985521984, 0.8916733985521984, 0.891..."
5,Lena.png,3,kapur_entropy,max,"[82, 126, 175]",4.046059,15.318348,1205.802904,34.724670,17.318042,0.704855,0.932681,"[[175, 175, 175, 175, 175, 175, 175, 175, 175,...","[3.813892126083374, 4.1644134521484375, 3.3120...","[15.318363633025005, 15.318363633025005, 15.31...","[1205.9774589538574, 1205.9774589538574, 1205....","[34.72718616522015, 34.72718616522015, 34.7271...","[17.31741170429817, 17.31741170429817, 17.3174...","[0.7049037330390853, 0.7049037330390853, 0.704...","[0.9327013076974919, 0.9327013076974919, 0.932..."
6,Lena.png,4,kapur_entropy,max,"[69, 105, 143, 181]",4.184656,18.003140,812.621152,28.492906,19.040226,0.749861,0.957079,"[[181, 181, 181, 181, 181, 181, 181, 181, 181,...","[6.428966283798218, 5.682740211486816, 4.68893...","[18.000676072721085, 17.99841674167613, 18.011...","[825.1025924682617, 773.6247367858887, 866.734...","[28.72459908281161, 27.81411038997812, 29.4403...","[18.965724091994986, 19.24550012929141, 18.751...","[0.7431879549736254, 0.7547505235645753, 0.746...","[0.9591724270754263, 0.9569271870575725, 0.958..."
7,Lena.png,5,kapur_en

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/GWO/GWO_kapur_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/GWO/GWO_kapur_dict.pkl', 'wb'))